In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import arviz as az
import pymc as pm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import attila_utils

In [2]:
fpath = '../../resources/QPS/5xFAD-mouse/CO28152_NFL-GFAP_datasum.xlsx'
dataw = pd.read_excel(fpath, sheet_name='NFL in vivo clean')
dataw['IRN'] = dataw.IRN.astype('str')

value_vars = [x for x in dataw.columns if re.match('^NF-L week \d+$', x)]
id_vars = [x for x in dataw.columns if not re.match('^NF-L week \d+$', x)]

df = dataw[value_vars].apply(lambda r: r - r.loc['NF-L week 0'], axis=1)
df = df.rename({x: 'ΔNfl week ' + re.sub('NF-L week ', '', x) for x in df.columns}, axis=1)
df['max ΔNfl'] = df.drop('ΔNfl week 0', axis=1).max(axis=1)

dataw = pd.concat([dataw, df], axis=1)
#dataw = dataw.set_index('IRN')
dataw

,Cohort,Group,IRN,Sex,Genotype,NF-L week 0,NF-L week 4,NF-L week 8,NF-L week 12,ΔNfl week 0,ΔNfl week 4,ΔNfl week 8,ΔNfl week 12,max ΔNfl
0,I,A,15117,f,tg/wt,576.0,861.0,757.0,687.0,0.0,285.0,181.0,111.0,285.0
1,I,A,15119,f,tg/wt,698.0,1042.0,783.0,644.0,0.0,344.0,85.0,-54.0,344.0
2,I,A,14627 A,f,tg/wt,701.0,1085.0,677.0,763.0,0.0,384.0,-24.0,62.0,384.0
3,III,A,15161,f,tg/wt,574.0,710.0,688.0,1028.0,0.0,136.0,114.0,454.0,454.0
4,II,A,15938,m,tg/wt,291.0,559.0,377.0,438.0,0.0,268.0,86.0,147.0,268.0
5,IV,A,16122,m,tg/wt,535.0,609.0,532.0,545.0,0.0,74.0,-3.0,10.0,74.0
6,I,B,15127,f,tg/wt,621.0,739.0,664.0,733.0,0.0,118.0,43.0,112.0,118.0
7,IV,B,15201,f,tg/wt,370.0,375.0,638.0,550.0,0.0,5.0,268.0,180.0,268.0
8,V,B,15245,f,tg/wt,678.0,593.0,688.0,577.0,0.0,-85.0,10.0,-101.0,10.0
9,I,B,15928,m,tg/wt,246.0,398.0,293.0,322.0,0.0,152.0,47.0,76.0,152.0


In [3]:
data = dataw.melt(value_vars=value_vars, id_vars=id_vars, value_name='Nfl', var_name='Week')
data['Week'] = data.Week.str.replace('NF-L week ', '').astype(float)
data

,Cohort,Group,IRN,Sex,Genotype,Week,Nfl
0,I,A,15117,f,tg/wt,0.0,576.0
1,I,A,15119,f,tg/wt,0.0,698.0
2,I,A,14627 A,f,tg/wt,0.0,701.0
3,III,A,15161,f,tg/wt,0.0,574.0
4,II,A,15938,m,tg/wt,0.0,291.0
...,...,...,...,...,...,...,...
231,V,J,15195 A,f,wt/wt,12.0,42.0
232,VI,J,15251,f,wt/wt,12.0,35.0
233,V,J,16180,m,wt/wt,12.0,24.0
234,V,J,16192,m,wt/wt,12.0,44.0


In [4]:
%connect_info

{
  "shell_port": 58865,
  "iopub_port": 58866,
  "stdin_port": 58867,
  "control_port": 58869,
  "hb_port": 58868,
  "ip": "127.0.0.1",
  "key": "33eb301b-acd03ca5b517293705f9e0a8",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": "python3",
  "jupyter_session": "/Users/jonesa7/CTNS/notebooks/2024-05-20-nfl-modeling/2024-05-20-nfl-modeling.ipynb"
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-7593b891-7a8b-4903-8de8-1c692e035d3c.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
